Showing how the framework can be loaded:

In [1]:
# built-in imports
import time

# package imports
from fiatmodel import Calibration

Testing the interface

In [2]:
# defining MESH calibration parameter bounds
class_dict_bounds = {
    1: {
        'sno': [0.0, 0.5],
        'rhos': [0.0, 0.5],
        'clay1': [10, 50],
        'sand1': [30, 70],
        'clay3': [20, 30],
    },
    3: {
        'lamx': [0.0, 1.0],
        'cmas': [0.0, 1.0],
        'clay2': [20, 60],
        'clay1': [0, 10],
    },
}

hydrology_dict_bounds = {
    1: {
        'zsnl': [0, 10],
        'zplg': [5, 15]
    },
    12: {
        'zpls': [0, 42],
        'zsnl': [0, 10],
        'zplg': [5, 15]
    },
}

routing_dict_bounds = {
    1: {
        'r2n': [0.0, 1.0],
        'r1n': [0.0, 1.0]
    },
    3: {
        'pwr': [0.0, 1.0],
        'flz': [0.0, 1.0]
    },
}

In [3]:
c = Calibration(
    calibration_software = 'ostrich',
    model_software = 'mesh',
    calibration_config = {
        'random_seed': int(time.time()),
        'algorithm': 'DDS',
        'algorithm_specs': { # refer to software manual for the keys
            'PerturbationValue': 0.2,
            'MaxIteration': 10_000,
            'UseRandomParamValue': None,
        },
        'dates': [ # one or more calibration dates
            {
                'start': '1985-01-01 00:00:00',
                'end': '2005-12-31 23:00:00',
            },
        ],
        'objective_functions': {
            'QO': {
                'kge_2012': ['-1 * station_1', '-1 * station_2', '(-0.5 * station_1) + (-0.5 * station_2)'],
                'nse': ['(-0.75 * station_1) + (-0.25 * station_2)', 'sin(station_1) + cos(station_2)'],
            },
        },
    },
    model_config = {
        'instance_path': './wolf-creek-research-basin/mesh/',
        'parameter_bounds': {
            'class': class_dict_bounds,
            'hydrology': hydrology_dict_bounds,
            'routing': routing_dict_bounds,
        },
        'executable': 'sa_mesh', # to be added to the copying stuff + required_files
    },
    observations = [
        {
            "name": "station_1",
            "type": "QO",
            "timeseries": [
                ("2020-01-01", 10.5),
                ("2020-01-02", 12.3),
            ],
            "unit": "m^3/s",
            "computational_unit": "subbasin",
            "computational_unit_id": 14,
            "freq": "1D",
        },
        {
            "name": "station_2",
            "type": "QO",
            "timeseries": [
                ("2020-01-01", 7.1),
                ("2020-01-03", 9.4),
            ],
            "unit": "m^3/s",
            "computational_unit": "subbasin",
            "computational_unit_id": 22,
            "freq": "1D",
        },
    ],
)

In [4]:
c.prepare(output_path='/Users/kasrakeshavarz/Downloads/test/')

/Users/kasrakeshavarz/Documents/github-repos/fiatmodel/src/fiatmodel/calibration/optimizer.py:194: UserWarning: The directory /Users/kasrakeshavarz/Downloads/test/ already exists. Contents may be overwritten.
  warnings.warn(f"The directory {path} already exists."
/Users/kasrakeshavarz/Documents/github-repos/fiatmodel/src/fiatmodel/calibration/optimizer.py:194: UserWarning: The directory /Users/kasrakeshavarz/Downloads/test/etc/templates already exists. Contents may be overwritten.
  warnings.warn(f"The directory {path} already exists."
/Users/kasrakeshavarz/Documents/github-repos/fiatmodel/src/fiatmodel/calibration/optimizer.py:194: UserWarning: The directory /Users/kasrakeshavarz/Downloads/test/etc already exists. Contents may be overwritten.
  warnings.warn(f"The directory {path} already exists."
/Users/kasrakeshavarz/Documents/github-repos/fiatmodel/src/fiatmodel/calibration/optimizer.py:194: UserWarning: The directory /Users/kasrakeshavarz/Downloads/test/etc/scripts already exists

In [5]:
import pandas as pd

ts_interval = pd.tseries.frequencies.to_offset
ts_interval('1h') == ts_interval("3600s")  # True

True